In [2]:
import psycopg2
import json

def enqueue_llm_request(user_input, task="summarize"):
    conn = psycopg2.connect("dbname=db user=user password=pw host=localhost")
    cur = conn.cursor()

    message = {
        "task": task,
        "input": user_input,
        "user_id": 42
    }

    cur.execute("SELECT * FROM pgmq_send('llm_queue', %s);", (json.dumps(message),))
    conn.commit()
    cur.close()
    conn.close()


In [3]:
import psycopg2
import json
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def process_llm_requests():
    conn = psycopg2.connect("dbname=db user=user password=pw host=localhost")
    cur = conn.cursor()

    while True:
        cur.execute("SELECT * FROM pgmq_read('llm_queue', 60, 3);")
        messages = cur.fetchall()

        for msg in messages:
            msg_id = msg[0]
            data = json.loads(msg[2])

            task = data["task"]
            text = data["input"]

            if task == "summarize":
                summary = summarizer(text, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
                print(f"Summary: {summary}")
                # می‌تونی این خلاصه رو در DB ذخیره کنی

            cur.execute("SELECT * FROM pgmq_delete('llm_queue', %s);", (msg_id,))
            conn.commit()

    cur.close()
    conn.close()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [4]:
!pip install transformers --quiet

from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

text = [
    "This project was incredibly useful and fun to implement.",
    "I regret using this product, it's terrible.",
    "The interface is smooth and very intuitive.",
    "Worst experience ever. Not recommended."
]

results = classifier(text)

for t, r in zip(text, results):
    print(f"Text: {t}\nLabel: {r['label']} | Score: {r['score']:.4f}\n")


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Text: This project was incredibly useful and fun to implement.
Label: POSITIVE | Score: 0.9998

Text: I regret using this product, it's terrible.
Label: NEGATIVE | Score: 0.9996

Text: The interface is smooth and very intuitive.
Label: POSITIVE | Score: 0.9998

Text: Worst experience ever. Not recommended.
Label: NEGATIVE | Score: 0.9998



In [5]:
!pip install transformers torch pandas --quiet

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

texts = [
    "I absolutely love the quality of this service.",
    "The design is outdated and the experience was bad.",
    "Fast delivery and excellent customer support.",
    "This is the worst thing I have ever bought.",
    "Decent product, could be better."
]

inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    outputs = model(**inputs)

scores = torch.nn.functional.softmax(outputs.logits, dim=1)
labels = torch.argmax(scores, dim=1)

label_map = {0: "NEGATIVE", 1: "POSITIVE"}
predictions = [label_map[label.item()] for label in labels]
confidences = scores.max(dim=1).values.tolist()

df = pd.DataFrame({
    "Text": texts,
    "Label": predictions,
    "Confidence": confidences
})

print(df)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.8 MB/s eta 0:00:00
                                                Text     Label  Confidence
0     I absolutely love the quality of this service.  POSITIVE    0.999878
1  The design is outdated and the experience was ...

#fastapi

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

app = FastAPI()

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

class TextInput(BaseModel):
    text: str

@app.post("/predict")
def predict_sentiment(input: TextInput):
    tokens = tokenizer(input.text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model(**tokens)
    probs = F.softmax(output.logits, dim=1)
    conf, pred = torch.max(probs, dim=1)
    label = "POSITIVE" if pred.item() == 1 else "NEGATIVE"
    return {"label": label, "confidence": float(conf.item())}
